Example of use of the Isochrones analysis.

We'll need to setup credentials that allow us to import a table and modify it; for convenience we'll used stored credentials, which you can save like this:

```python
from cartoframes.auth import Credentials
Credentials(username='...', api_key='...').save()
```

In [1]:
from cartoframes.auth import set_default_credentials, Credentials
credentials = Credentials.from_file()
set_default_credentials(credentials)

We'll use a dataset with some locations:

In [2]:
from cartoframes.data import Dataset
dataset = Dataset("SELECT * FROM carto_places WHERE country='Spain'")

First we need an instace of the Isolines service class (note that the default credentials will be used to access the service):

In [3]:
from cartoframes.data.services import Isolines
iso = Isolines()

We'll compute three isochrones for each point, corresponding to 20, 30 and 60 minute drives (given in seconds). But before doing any actual processing we'll perform a *dry run* to learn how many credits it would cost us to the job:

In [4]:
estimate = iso.isochrones(dataset, [1200,2400,3600], mode='car', dry_run=True).metadata
estimate

{'required_quota': 6}

Let's check if we have enough quota for the operation by using the `available_quota` method:

In [5]:
print(estimate.get('required_quota')<=iso.available_quota())

True


Since we intend to visualize the result in a map, we'll add the `exclusive` parameter to generate non-overlapping areas corresponding to _exclusive_ interval ranges for 0 to 1200, 1200 to 2400 and 2400 to 3600 seconds. Otherwise areas would overlap and smaller areas may be obscured by the larger ones in the map.

Note that, in order to the estimate quota usage from the dry run to be applicable, we should not change the dataset or ranges argumente in the final call.

In [6]:
isochrones = iso.isochrones(dataset, [1200,2400,3600], mode='car', exclusive=True).data
isochrones.dataframe

,cartodb_id,source_id,data_range,lower_data_range,the_geom
cartodb_id,,,,,
1,1,2.0,1200.0,0.0,0106000020E61000000100000001030000000100000033...
2,2,2.0,2400.0,1200.0,0103000020E61000000200000033000000A774B0FECFA1...
3,3,2.0,3600.0,2400.0,0106000020E61000000400000001030000000100000013...
4,4,4.0,1200.0,0.0,0106000020E61000000100000001030000000100000033...
5,5,4.0,2400.0,1200.0,0106000020E61000000500000001030000000100000019...
6,6,4.0,3600.0,2400.0,0106000020E61000000400000001030000000100000013...


Now let's see how it actually looks in a map, but first we'll add a column named `range_label` with properly formatted labels for the data areas (showing the time interval in minutes). Then we'll use this new column as a category for the visualization.


In [7]:
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import color_category_layer
isochrones.dataframe['range_label'] = isochrones.dataframe.apply(lambda r: '%.0f to %.0f min.' % (r['lower_data_range']/60, r['data_range']/60), axis=1)
Map([color_category_layer(isochrones, 'range_label', palette='purpor', title='Isochrone Areas'), Layer(dataset)])